In [1]:
# Tons and tons of imports!
# from collections import Counter, namedtuple
# from itertools import chain
# import json
# import math
# import os
# from pathlib import Path
# from tqdm.notebook import tqdm, trange
# from typing import List, Tuple, Dict, Set, Union
# import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# import torch
# import torch.nn as nn
# from torch.nn import init
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
# import torch.nn.utils
# import torch.nn.functional as F
# from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

**Load Data**

In [2]:
LF_train = pd.read_csv('./raw_data/LF_train.csv')
LH_train = pd.read_csv('./raw_data/LH_train.csv')
# rf_train = pd.read_csv('./raw_data/RF_train.csv')
# rf_test = pd.read_csv('./draw_ata/RH_train.csv')

In [3]:
### turn literals into numericals
LF_train = LF_train.drop(columns=['dob', 'gait', 'Gait'])
LF_train['forceplate_date'] = LF_train['forceplate_date'].str.replace("-","").astype(int)

### nan value
LF_train = LF_train.fillna(0)
LF_train = LF_train.replace('Not able to trot', 0)
LF_train = LF_train.replace('Not able to walk', 0)
LF_train = LF_train.replace('no data', 0)
LF_train['Speed'] = LF_train['Speed'].astype(float)

for col in LF_train.columns:
    if 'V' in col or 'Speed' in col :
        LF_train[col] = LF_train[col].replace(0, LF_train[col].mean())

### sort by id
LF_train = LF_train.sort_values(by=['id'])

### perform PCA and get rid of redundant columns

LF_train.to_csv('LF_train.csv')
LF_train

,id,gender,weight,forceplate_date,age,LF,speed,V1_LF,V1_RF,V1_LH,...,V28_LH_trot,V28_RH_trot,V29_LF_trot,V29_RF_trot,V29_LH_trot,V29_RH_trot,V30_LF_trot,V30_RF_trot,V30_LH_trot,V30_RH_trot
67,2,0,34.600,20190626,5.079452,1,1.315,183.831519,261.735816,173.086274,...,0.200878,0.255695,0.378632,0.375222,0.136255,0.109891,0.250000,0.272321,0.232143,0.245536
76,3,1,25.600,20191003,7.178082,0,1.1400000000000001,178.404307,194.485630,115.977757,...,0.176045,0.163725,0.283679,0.339011,0.198281,0.179028,0.291317,0.282913,0.212885,0.212885
94,6,0,36.700,20110425,1.772603,1,1.032,248.316618,245.881475,150.944727,...,0.227326,0.193144,0.249856,0.313535,0.238719,0.197890,0.269729,0.254697,0.246764,0.228810
54,7,0,64.411,20090715,5.608219,0,0.962,375.960740,373.157890,221.110528,...,0.220833,0.146660,0.272691,0.348814,0.238568,0.139927,0.268178,0.260623,0.228990,0.242210
11,8,0,36.400,20190909,0.783562,0,1.1066666666666667,251.456723,251.681851,134.528131,...,0.178055,0.201276,0.465763,0.421552,0.183823,0.112685,0.301775,0.287968,0.177515,0.232742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,176,0,27.200,20080206,3.602740,0,1.034,172.448409,169.714560,138.505126,...,0.249825,0.232572,0.247507,0.254186,0.264056,0.234251,0.290997,0.290997,0.205252,0.212755
44,178,0,28.200,20090211,1.978082,0,1.1575,195.628736,200.532297,131.060970,...,0.214868,0.210482,0.279966,0.287559,0.215779,0.216696,0.286990,0.283163,0.211735,0.218112
92,180,1,26.300,20080609,3.942466,0,1.0779999999999998,169.406911,172.968121,110.135375,...,0.226625,0.219313,0.254491,0.264647,0.243868,0.236994,0.274532,0.275099,0.230289,0.220079
53,181,1,20.200,20090226,8.986301,0,1.188,128.704239,127.564176,81.999105,...,0.187021,0.197141,0.303722,0.298653,0.204484,0.193141,0.263977,0.277233,0.228242,0.230548


In [4]:
### turn literals into numericals
LH_train = LH_train.drop(columns=['dob', 'gait', 'Gait'])
LH_train['forceplate_date'] = LH_train['forceplate_date'].str.replace("-","").astype(int)

### nan value
LH_train = LH_train.fillna(0)
# LH_train = LH_train.replace('Not able to trot', 0)
# LH_train['Speed'] = LH_train['Speed'].astype(float)

# for col in LH_train.columns:
#     if 'V' in col or 'Speed' in col :
#         LH_train[col] = LH_train[col].replace(0, LH_train[col].mean())

### sort by id
LH_train = LH_train.sort_values(by=['id'])

### perform PCA and get rid of redundant columns

LH_train.to_csv('LH_train.csv')
LH_train

,id,gender,weight,forceplate_date,age,LH,speed,V1_LF,V1_RF,V1_LH,...,V28_LH_trot,V28_RH_trot,V29_LF_trot,V29_RF_trot,V29_LH_trot,V29_RH_trot,V30_LF_trot,V30_RF_trot,V30_LH_trot,V30_RH_trot
5,1,1,24.700,20170510,10.558904,1,0,0.000000,0.000000,0.000000,...,0.193317,0.213219,0.293557,0.312367,0.188542,0.205534,0.286856,0.287958,0.214935,0.210251
6,3,1,25.600,20191003,7.178082,1,1.1400000000000001,178.404307,194.485630,115.977757,...,0.176045,0.163725,0.283679,0.339011,0.198281,0.179028,0.291317,0.282913,0.212885,0.212885
87,5,0,36.200,20110307,8.391781,1,1.12,267.060125,266.926251,169.227812,...,0.196803,0.183123,0.313062,0.298513,0.195874,0.192550,0.277131,0.275904,0.232373,0.214592
92,7,0,64.411,20090715,5.608219,1,0.962,375.960740,373.157890,221.110528,...,0.220833,0.146660,0.272691,0.348814,0.238568,0.139927,0.268178,0.260623,0.228990,0.242210
39,8,0,36.400,20190909,0.783562,1,1.1066666666666667,251.456723,251.681851,134.528131,...,0.000000,0.201276,0.465763,0.421552,0.000000,0.112685,0.301775,0.287968,0.177515,0.232742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,177,1,20.100,20090305,3.323288,0,1.16,148.599453,142.968998,97.123233,...,0.192578,0.208536,0.313399,0.281439,0.201820,0.203343,0.281919,0.284490,0.221080,0.212511
64,181,1,20.200,20090226,8.986301,0,1.188,128.704239,127.564176,81.999105,...,0.187021,0.197141,0.303722,0.298653,0.204484,0.193141,0.263977,0.277233,0.228242,0.230548
57,182,1,32.700,20080429,2.326027,0,1.0466666666666666,201.934578,201.288505,147.452671,...,0.000000,0.453658,0.000000,0.530241,0.000000,0.469759,0.251149,0.306789,0.191935,0.250128
41,183,0,36.500,20070716,3.208219,0,1,235.293254,251.611638,148.724997,...,0.203955,0.211797,0.298878,0.272637,0.209835,0.218650,0.273709,0.278511,0.226090,0.221689


In [5]:
# split by walk / trot
trot_list = []
name_list = []
for col in LF_train.columns:
    if 'trot' in col:
        trot_list.append(col)

        new_name = col.split('_trot')
        name_list.append(new_name[0])

LF_train_trot = pd.DataFrame(LF_train, columns = trot_list)
LF_train_trot.columns = name_list

both = ['id', 'gender', 'weight', 'forceplate_date', 'speed', 'age', 'Speed', 'LF']
LF_train_both = pd.DataFrame(LF_train, columns = both)

# add binary col is_trot
LF_train_trot = LF_train_both.join(LF_train_trot)
LF_train_trot['is_trot'] = 1

/var/folders/69/r1dg6qsd2_j9csr9658538qw0000gn/T/ipykernel_63581/1214989995.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  LF_train_trot['is_trot'] = 1


In [6]:
LF_train_walk = LF_train.drop(columns=trot_list)
LF_train_walk['is_trot'] = 0

/var/folders/69/r1dg6qsd2_j9csr9658538qw0000gn/T/ipykernel_63581/3765687609.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  LF_train_walk['is_trot'] = 0


In [7]:
# concat walk and trot
LF_train = pd.concat([LF_train_walk, LF_train_trot])

In [8]:
### nan value
LF_train = LF_train.fillna(0)
LF_train = LF_train.replace('Not able to trot', 0)
LF_train['Speed'] = LF_train['Speed'].astype(float)

for col in LF_train.columns:
    if 'V' in col or 'Speed' in col :
        LF_train[col] = LF_train[col].replace(0, LF_train[col].mean())

### sort by id
LF_train = LF_train.sort_values(by=['id'])

### perform PCA and get rid of redundant columns

LF_train.to_csv('LF_train.csv')
LF_train

,id,gender,weight,forceplate_date,age,LF,speed,V1_LF,V1_RF,V1_LH,...,V29_LF,V29_RF,V29_LH,V29_RH,V30_LF,V30_RF,V30_LH,V30_RH,Speed,is_trot
67,2,0,34.6,20190626,5.079452,1,1.315,183.831519,261.735816,173.086274,...,0.354105,0.485929,0.075465,0.084501,0.235294,0.249135,0.262976,0.252595,1.710,0
67,2,0,34.6,20190626,5.079452,1,1.315,185.208712,249.535675,145.922523,...,0.378632,0.375222,0.136255,0.109891,0.250000,0.272321,0.232143,0.245536,1.710,1
76,3,1,25.6,20191003,7.178082,0,1.1400000000000001,178.404307,194.485630,115.977757,...,0.402185,0.492270,0.076547,0.028998,0.260504,0.266106,0.233894,0.239496,1.812,0
76,3,1,25.6,20191003,7.178082,0,1.1400000000000001,280.954561,290.108186,144.419058,...,0.283679,0.339011,0.198281,0.179028,0.291317,0.282913,0.212885,0.212885,1.812,1
94,6,0,36.7,20110425,1.772603,1,1.032,248.316618,245.881475,150.944727,...,0.303288,0.328469,0.177950,0.190293,0.250408,0.261827,0.259584,0.228181,2.026,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,180,1,26.3,20080609,3.942466,0,1.0779999999999998,169.406911,172.968121,110.135375,...,0.281286,0.276961,0.196854,0.244898,0.264371,0.250388,0.239254,0.245987,2.030,0
53,181,1,20.2,20090226,8.986301,0,1.188,211.831023,212.581617,131.925549,...,0.303722,0.298653,0.204484,0.193141,0.263977,0.277233,0.228242,0.230548,2.100,1
53,181,1,20.2,20090226,8.986301,0,1.188,128.704239,127.564176,81.999105,...,0.346563,0.344794,0.157216,0.151427,0.262001,0.265471,0.234818,0.237710,2.100,0
48,184,0,22.6,20090128,1.890411,0,1.1883333333333332,157.895443,156.617448,102.073996,...,0.424436,0.396303,0.109802,0.069459,0.252557,0.272041,0.238675,0.236727,2.142,0
